<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/motion/motion_diffusion_unity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/GuyTevet/motion-diffusion-model

In [ ]:
!nvidia-smi

In [ ]:
#@title colab
!pip install git+https://github.com/openai/CLIP.git
!pip install smplx chumpy

In [ ]:
# motion-diffusion-modelのクローン
!git clone https://github.com/GuyTevet/motion-diffusion-model
!mkdir motion-diffusion-model/save

In [ ]:
!git clone https://github.com/EricGuo5513/HumanML3D.git
!unzip ./HumanML3D/HumanML3D/texts.zip -d ./HumanML3D/HumanML3D/
!cp -r HumanML3D/HumanML3D motion-diffusion-model/dataset/HumanML3D

In [ ]:
# motion-diffusion-model/prepare/download_smpl_files.sh
"""
mkdir -p body_models
cd body_models/

echo -e "The smpl files will be stored in the 'body_models/smpl/' folder\n"
gdown "https://drive.google.com/uc?id=1INYlGA76ak_cKGzvpOV2Pe6RkYTlXTW2&confirm=t&uuid=a9f224ab-be71-4056-9054-e9fbe6867da1&at=ALgDtsycjLnL_7sKC4uzs53YVngh:1675731179916"
rm -rf smpl

unzip smpl.zip
echo -e "Cleaning\n"
rm smpl.zip

echo -e "Downloading done!"
"""

In [ ]:
%cd motion-diffusion-model
!bash prepare/download_smpl_files.sh

In [ ]:
# download pretrained model
!gdown "https://drive.google.com/u/0/uc?id=1PE0PK8e5a5j-7-Xhs5YET5U5pGh0c821&export=download&confirm=t&uuid=13554310-f2ad-42fa-b31d-2aae29cb6b04&at=ALgDtsziu0_n1GsmHSVSGgBO08pq:1675731368572"

In [ ]:
%cd /content/motion-diffusion-model/save
!cp '/content/motion-diffusion-model/humanml_trans_enc_512.zip' .
!unzip humanml_trans_enc_512.zip
!rm -rf humanml_trans_enc_512.zip
%cd /content/motion-diffusion-model/

In [ ]:
!ls

In [ ]:
#@title infernece with test set prompts
!python -m sample.generate --model_path /content/motion-diffusion-model/save/humanml_trans_enc_512/model000200000.pt --num_samples 10 --num_repetitions 3

In [ ]:
#@title inference with text file
!python -m sample.generate --model_path /content/motion-diffusion-model/save/humanml_trans_enc_512/model000200000.pt --input_text /content/motion-diffusion-model/assets/example_text_prompts.txt

In [ ]:
#@title inference with single prompt
!python -m sample.generate --model_path /content/motion-diffusion-model/save/humanml_trans_enc_512/model000200000.pt --text_prompt "the person walked forward and is picking up his toolbox." --motion_length 9.0

In [ ]:
#@title create pose file for unity
# results.npyの中身の確認
import numpy as np
prompt = "the person walked forward and is picking up his toolbox."
ans = prompt.replace(" ", "_")[:-1]
prefix = "/content/motion-diffusion-model/save/humanml_trans_enc_512/samples_humanml_trans_enc_512_000200000_seed10_"
result_file = prefix + ans + "/results.npy"
data = np.load(result_file, allow_pickle=True)
data

In [29]:
data = np.load(result_file, allow_pickle=True)
dataDict = dict(enumerate(data.flatten()))[0]
dataDict["motion"] = dataDict["motion"].tolist()
dataDict["lengths"] = dataDict["lengths"].tolist()
print(dataDict)

{'motion': [[[[0.0, 0.002438301220536232, 0.005258851684629917, 0.008486133068799973, 0.012164638377726078, 0.016323929652571678, 0.021023105829954147, 0.02612582966685295, 0.03133036941289902, 0.036125700920820236, 0.039924256503582, 0.04250049963593483, 0.04392693564295769, 0.04418506100773811, 0.043218836188316345, 0.04128145053982735, 0.0382794626057148, 0.03384220227599144, 0.02794927917420864, 0.02057620882987976, 0.01225807424634695, 0.003840816207230091, -0.00369366817176342, -0.009539072401821613, -0.01308692991733551, -0.013842365704476833, -0.01150155533105135, -0.006128206849098206, 0.0016849583480507135, 0.011124052107334137, 0.02149219438433647, 0.032242074608802795, 0.042849041521549225, 0.05280011147260666, 0.061558522284030914, 0.0688740536570549, 0.07488558441400528, 0.07987409830093384, 0.08406786620616913, 0.08762962371110916, 0.0906599685549736, 0.09325535595417023, 0.09549460560083389, 0.09743297100067139, 0.09912046790122986, 0.10057403892278671, 0.10185112059116

In [ ]:
# サイズの確認
print("repetitions:", len(dataDict["motion"])) # 繰り返し
print("joint:", len(dataDict["motion"][0]))  # 関節
print("xyz:", len(dataDict["motion"][0][0])) # XYZ
print("frame:", len(dataDict["motion"][0][0][0])) # フレーム

In [31]:
# results.jsonに出力
import json
with open("/content/pose_results.json", mode='w') as f:
    json.dump(dataDict, f)

In [ ]:
# 連結してmotionを作成し、resultのなかに書き込んでいって連関した動きを生成できるようにする。

In [15]:
#@title display video

video_path = "/content/motion-diffusion-model/save/humanml_trans_enc_512/samples_humanml_trans_enc_512_000200000_seed10/samples_00_to_06.mp4"  #@param

# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open(video_path, 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

 